In [ ]:
cd(@__DIR__)
using Pkg; Pkg.activate("."); Pkg.instantiate()

using Revise

using DataFrames, Dates, Statistics
using DelimitedFiles, CSV

using OrdinaryDiffEq, DiffEqFlux, Flux
using SciMLSensitivity

using Optimization, BlackBoxOptim
using Zygote

using Interpolations

using Random
Random.seed!(123)


In [12]:

# ===========================================================
# USER INPUT:

# set data directory
data_path = joinpath(pwd(),"basin_dataset_public_v1p2")

# choose model M50 or M100
chosen_model_id = "M100"

# choose basin id
basin_id = "01013500"

# define training and testing period
train_start_date = Date(1980,10,01)
train_stop_date = Date(1981,09,30)
test_start_date = Date(1981,10,01)
test_stop_date = Date(2010,09,30)


# if `false`, read the bucket model (M0) parameters from "bucket_opt_init.csv"
train_bucket_model = false

# ===========================================================

includet("HydroNODE_data.jl")
includet("HydroNODE_models.jl")
includet("HydroNODE_training.jl")


# -------------------------------------------------------
# Objective function: Nash-Sutcliffe Efficiency

NSE(pred, obs) = 1 - sum((pred .- obs).^2) / sum((obs .- mean(obs)).^2)

function NSE_loss(pred_model, params, batch, time_batch)

    pred, = pred_model(params, time_batch)
    loss = -NSE(pred,batch)

    return loss, pred
end

# -------------------------------------------------------
# Load and preprocess data

input_var_names = ["Daylight(h)", "Prec(mm/day)", "Tmean(C)"]
output_var_name = "Flow(mm/s)"

df = load_data(lpad(string(basin_id), 8, "0"), data_path)

# drop unused cols
select!(df, Not(Symbol("SWE(mm)")));
select!(df, Not(Symbol("Tmax(C)")));
select!(df, Not(Symbol("Tmin(C)")));

# adjust start and stop date if necessary
if df[1, "Date"] != train_start_date
    train_start_date = maximum([df[1, "Date"],train_start_date])
end

if df[end, "Date"] != test_stop_date
    test_stop_date = minimum([df[end, "Date"], test_stop_date])
end

# format data
data_x, data_y, data_timepoints,
train_x, train_y, train_timepoints, = prepare_data(df,
(train_start_date, train_stop_date, test_start_date, test_stop_date),input_var_names,output_var_name)

# normalize data
norm_moments_in = [mean(data_x, dims=1); std(data_x, dims = 1)]

norm_P = prep_norm(norm_moments_in[:,2])
norm_T = prep_norm(norm_moments_in[:,3])

# -------------------------------------------------------
# interpolation

itp_method = SteffenMonotonicInterpolation()

itp_Lday = interpolate(data_timepoints, data_x[:,1], itp_method)
itp_P = interpolate(data_timepoints, data_x[:,2], itp_method)
itp_T = interpolate(data_timepoints, data_x[:,3], itp_method)



# adjust start and stop date if necessary
if df[1, "Date"] != train_start_date
    train_start_date = maximum([df[1, "Date"],train_start_date])
end

if df[end, "Date"] != test_stop_date
    test_stop_date = minimum([df[end, "Date"], test_stop_date])
end

# format data
data_x, data_y, data_timepoints,
train_x, train_y, train_timepoints, = prepare_data(df,(train_start_date, train_stop_date, test_start_date, test_stop_date),input_var_names,output_var_name)

# normalize data
norm_moments_in = [mean(data_x, dims=1); std(data_x, dims = 1)]

norm_P = prep_norm(norm_moments_in[:,2])
norm_T = prep_norm(norm_moments_in[:,3])

# -------------------------------------------------------
# interpolation

itp_method = SteffenMonotonicInterpolation()

itp_Lday = interpolate(data_timepoints, data_x[:,1], itp_method)
itp_P = interpolate(data_timepoints, data_x[:,2], itp_method)
itp_T = interpolate(data_timepoints, data_x[:,3], itp_method)






Basin ID: 01013500

Hydrologic unit code (HUC): 01

Forcings data set: daymet

Basin Data from forcings file:
Lat,    Elev,   Area [km^2]
[46.84, 353.0, 2260.09]

Gauging station name: 
                  Fish River near Fort Kent, Maine

 Latitude,  Longitude    
[47.23739 -68.58264]

Input variables: 
["Daylight(h)", "Prec(mm/day)", "Tmean(C)"]

Output variable: 
Flow(mm/s)

Size: train_x=(365, 3) // train_y=(365,)
Size: test_x= (10592, 3) //  test_y= (10592,)

Input variables: 
["Daylight(h)", "Prec(mm/day)", "Tmean(C)"]

Output variable: 
Flow(mm/s)

Size: train_x=(365, 3) // train_y=(365,)
Size: test_x= (10592, 3) //  test_y= (10592,)



10957-element interpolate(::Vector{Float64}, ::Vector{Float64}, SteffenMonotonicInterpolation()) with element type Float64:
  6.08
 10.53
 11.835
  7.380000000000001
  4.8
  5.41
  6.405000000000001
  7.675000000000001
  5.48
  3.67
  1.5300000000000002
  5.555
  4.35
  ⋮
 11.5
  9.91
  8.41
 11.175
 10.585
  8.035
  9.09
  9.485
 10.685000000000002
 14.13
 16.215
 15.435

In [13]:
train_y

365-element Vector{Float64}:
 0.5509980741595841
 0.5607406727203627
 0.5585756508179675
 0.6711567897425189
 0.821625811958987
 0.8670912719092866
 0.9114742209083887
 0.9482795932491075
 0.9634347465658741
 0.9504446151515028
 0.9612697246634788
 1.0825109511976114
 1.1366364987574917
 ⋮
 0.7534276220335374
 0.7610051986919207
 0.7447675344239565
 0.7220348044488066
 0.7068796511320403
 0.8259558557637774
 0.8595136952509034
 0.8681737828604843
 0.8854939580796461
 0.9114742209083887
 0.8854939580796461
 0.8508536076413225

In [ ]:
function ConvEncoder(; imgsize=(365,1,1), nclasses=27) 
    out_conv_size = (imgsize[1]÷4 - 3, 1, 16)
    
    return Chain(
            Conv((5, 1), imgsize[end]=>8, relu),
            MaxPool((2, 1)),
            Conv((5, 1), 6=>16, relu),
            MaxPool((2, 1)),
            flatten,
            Dense(prod(out_conv_size), 128, relu), 
            Dense(128, 64, relu), 
            Dense(64, nclasses)
          )
end